In [2]:
import cv2
import mediapipe as mp
# import csv
import numpy as np
import time
import torch
import math

2024-04-06 19:17:06.294347: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 19:17:07.159411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
# cap = cv2.VideoCapture(0)
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
    
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         # image.flags.writable = False

#         results = pose.process(image)

#         # image.flags.writable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

#         cv2.imshow('Mediapipe feed', image)
        
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows()

In [5]:
print(mp_pose.POSE_CONNECTIONS)

frozenset({(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17), (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4), (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3), (11, 12), (27, 29), (13, 15)})


In [6]:
print(mp.solutions.pose.POSE_CONNECTIONS)

frozenset({(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17), (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4), (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3), (11, 12), (27, 29), (13, 15)})


In [7]:
def body_in_frame(landmarks):
    left = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y
    right = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y
    nose = landmarks[mp_pose.PoseLandmark.NOSE.value].y

    # heel out of frame
    if left > 0.95 or right > 0.95:
        return False
    # heel wayy to in frame
    if left < 0 or right < 0 or nose < 0:
        return False
    if nose > 0.15:
        return False

    return True

In [8]:
cap = cv2.VideoCapture(0)
captured_body = False
quit = False
measurements = []
start = None
image = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            
            # variables to maintain continous stream of data
            if not captured_body:
                captured_body = True
                start = time.time() 

            # if our body isn't in frame, restart
            if not body_in_frame(landmarks):
                captured_body = False
                start = None
                measurements.clear()
                
            # add landmarks to our current stuff
            if captured_body:
                measurements.append(landmarks)
                
                # quit when we have 3 seconds of data
                if time.time() - start > 3:
                    quit = True
            
        except:
            captured_body = False
            start = None
            measurements.clear()

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('Mediapipe feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q') or quit:
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1712456229.331472   71823 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712456229.357770   71896 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce RTX 2060/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Turn measurements into a 3d tensor. 
The tensor should be of shape(`len(measurements)`, `33` (the number of landmarks), `4` (x, y, z, and visisibility))

In [9]:
tensor_data = torch.zeros(len(measurements), 33, 4)
print(tensor_data.shape)

for idx, itm in enumerate(measurements):
    for jdx, jtm in enumerate(itm):
        tensor_data[idx][jdx][0] = jtm.x
        tensor_data[idx][jdx][1] = jtm.y
        tensor_data[idx][jdx][2] = jtm.z
        tensor_data[idx][jdx][3] = jtm.visibility 

torch.Size([65, 33, 4])


In [ ]:
cv2.imshow('BGR Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
avg_tensor = torch.mean(tensor_data[:, :, :], dim=0)
for i,tns in enumerate(avg_tensor):
    print(tns, i)

tensor([ 0.4823,  0.0581, -0.4958,  0.9989]) 0
tensor([ 0.4909,  0.0398, -0.4750,  0.9988]) 1
tensor([ 0.4962,  0.0405, -0.4750,  0.9989]) 2
tensor([ 0.5013,  0.0413, -0.4751,  0.9989]) 3
tensor([ 0.4755,  0.0383, -0.4754,  0.9986]) 4
tensor([ 0.4701,  0.0382, -0.4754,  0.9986]) 5
tensor([ 0.4650,  0.0382, -0.4754,  0.9986]) 6
tensor([ 0.5093,  0.0497, -0.3149,  0.9990]) 7
tensor([ 0.4574,  0.0477, -0.3189,  0.9990]) 8
tensor([ 0.4915,  0.0776, -0.4337,  0.9990]) 9
tensor([ 0.4713,  0.0772, -0.4347,  0.9988]) 10
tensor([ 0.5514,  0.1670, -0.1916,  0.9997]) 11
tensor([ 0.4085,  0.1650, -0.1875,  0.9996]) 12
tensor([ 0.5962,  0.2997, -0.1281,  0.9886]) 13
tensor([ 0.3577,  0.3028, -0.1252,  0.9767]) 14
tensor([ 0.6489,  0.4011, -0.2903,  0.9870]) 15
tensor([ 0.2990,  0.4072, -0.2688,  0.9760]) 16
tensor([ 0.6686,  0.4349, -0.3291,  0.9735]) 17
tensor([ 0.2738,  0.4408, -0.3069,  0.9617]) 18
tensor([ 0.6660,  0.4340, -0.3914,  0.9759]) 19
tensor([ 0.2751,  0.4387, -0.3731,  0.9649]) 20
te

In [12]:
torch.save(avg_tensor, 'avg_tensor.pt')

In [28]:
# pair is the two landmarks we want to measure between
def get_distance_between_two_t(avg_tensor, pair):
    l1, l2 = pair[0], pair[1]
    
    i1_x = avg_tensor[l1][0]
    i1_y = avg_tensor[l1][1]
    i2_x = avg_tensor[l2][0]
    i2_y = avg_tensor[l2][1]

    return math.sqrt((i1_x - i2_x)**2 + (i1_y - i2_y)**2)

In [14]:
def get_distance_t(avg_tensor, landmarks):
    distance = 0
    
    for i in range(len(landmarks) - 1):  # Iterate up to the second last element
        pair = (landmarks[i], landmarks[i+1])  # Access current element and the next one
        distance += get_distance_between_two_t(avg_tensor, pair)

    return distance

In [15]:
def base_neck_tensor(avg_tensor):
    base = torch.zeros(4)
    base[0] = (avg_tensor[mp_pose.PoseLandmark.LEFT_SHOULDER.value][0] + avg_tensor[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0]) / 2
    avg_y_mouth = (avg_tensor[mp_pose.PoseLandmark.MOUTH_LEFT.value][1] + avg_tensor[mp_pose.PoseLandmark.MOUTH_RIGHT.value][1]) / 2
    avg_y_shoulder = (avg_tensor[mp_pose.PoseLandmark.LEFT_SHOULDER.value][1] + avg_tensor[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1]) / 2
    base[1] = (avg_y_mouth + avg_y_shoulder) / 2
    
    # print(base)
    return base

In [16]:
def find_sleeve(avg_tensor):
    # print(avg_tensor)
    d1 = get_distance_t(avg_tensor, [mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_ELBOW.value, mp_pose.PoseLandmark.LEFT_WRIST.value])
    d2 = get_distance_t(avg_tensor, [mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_ELBOW.value, mp_pose.PoseLandmark.RIGHT_WRIST.value])

    base_tensor = base_neck_tensor(avg_tensor)
    print(base_tensor)

    d1 += math.sqrt((base_tensor[0] - avg_tensor[mp_pose.PoseLandmark.LEFT_SHOULDER.value][0])**2 + (base_tensor[1] - avg_tensor[mp_pose.PoseLandmark.LEFT_SHOULDER.value][1])**2)
    d2 += math.sqrt((base_tensor[0] - avg_tensor[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0])**2 + (base_tensor[1] - avg_tensor[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1])**2)

    # print(d1, d2)
    
    # print(d1, d2)
    return (d1 + d2) / 2

In [17]:
def find_inseam():
    d1 = get_distance_t(avg_tensor, [mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value])
    d2 = get_distance_t(avg_tensor, [mp_pose.PoseLandmark.RIGHT_HIP.value, mp_pose.PoseLandmark.RIGHT_KNEE.value, mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    # print(d1, d2)
    return (d1 + d2) / 2

In [18]:
def find_height():
    avg_bottom_x = (avg_tensor[mp_pose.PoseLandmark.LEFT_ANKLE.value][0] + avg_tensor[mp_pose.PoseLandmark.RIGHT_ANKLE.value][0]) / 2
    avg_bottom_y = (avg_tensor[mp_pose.PoseLandmark.LEFT_ANKLE.value][1] + avg_tensor[mp_pose.PoseLandmark.RIGHT_ANKLE.value][1]) / 2
    nose = mp_pose.PoseLandmark.NOSE.value
    
    return math.sqrt((avg_tensor[nose][0] - avg_bottom_x)**2 + (avg_tensor[nose][1] - avg_bottom_y)**2)

In [ ]:
height = 74
fact = (height - 6) / find_height()

In [ ]:
find_sleeve(avg_tensor) * fact

In [ ]:
find_inseam() * fact

In [ ]:
find_height() 

In [ ]:
def find_chest():
    return get_distance_t(avg_tensor, [mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_SHOULDER.value])

In [24]:
def find_waist():
    return get_distance_t(avg_tensor, [mp_pose.PoseLandmark.RIGHT_HIP.value, mp_pose.PoseLandmark.LEFT_HIP.value])

In [25]:
find_chest() * fact * 2

26.05870832317778

In [26]:
find_waist() * fact

7.403432427980388